In [ ]:
import sys
sys.path.append("/public/home/renkan/zyy/chronos-forecasting")
print(sys.path)

In [ ]:
!nvidia-smi

In [ ]:
import torch
import json
import argparse
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.distributed as dist
from tqdm import tqdm, trange
from data_provider.metrics import metric

In [ ]:
f_mse = open("/public/home/renkan/zyy/AutoTimes/forecast_log/ETTh1_mse.jsonl", 'r', encoding='utf-8')
count = 1
data_mse = []
for line_mse in f_mse:
	if count % 2 == 0:
		data_mse.extend(json.loads(line_mse))
	count += 1
# plt.figure(figsize=(20, 5))
# plt.plot(data_mse)
# plt.hist(data_mse, bins=100, density=True, histtype="step")
plt.hist(data_mse, bins=100)
plt.show()

In [ ]:
f_pred = open("/public/home/renkan/cty/chronos-forecasting/scripts/test/forecast_log/ETTh1_original_5.jsonl", 'r', encoding='utf-8')
f_gt = open("/public/home/renkan/cty/chronos-forecasting/scripts/test/forecast_log/ETTh1_gt_5.jsonl", 'r', encoding='utf-8')
#f_mean = open("/public/home/renkan/cty/chronos-forecasting/scripts/test/forecast_log/ETTh1_mean_5.jsonl", 'r', encoding='utf-8')
count=1
x = np.linspace(0,24,24)
for line_pred,line_gt in zip(f_pred,f_gt):
    data_pred = json.loads(line_pred)
    data_gt = json.loads(line_gt)
    for i in range(len(data_pred)):
        pred = np.array(data_pred[i])
        data_gt = np.array(data_gt)
        plt.plot(x,pred,'b',label='prediction')
        plt.plot(x,data_gt,'r',label='gt')
        plt.legend()
        #plt.title(f"mse_chronos:{mse},mse_PatchTST:{mse_b}")
        if count%5==0:
            plt.savefig(f'/public/home/renkan/cty/chronos-forecasting/output/test_results_zscore_pictures_ETTh1/ETTh1_{count}', dpi=300)
        count += 1
        plt.show()
        plt.close()

In [ ]:

# f_pred = open("/public/home/renkan/hwc/exp/chronos-forecasting/forecast_log/Solar_original_5.jsonl", 'r', encoding='utf-8')
# f_gt = open("/public/home/renkan/hwc/exp/chronos-forecasting/forecast_log/Solar_gt_5.jsonl", 'r', encoding='utf-8')
# f_mean = open("/public/home/renkan/hwc/exp/chronos-forecasting/forecast_log/Solar_mean_5.jsonl", 'r', encoding='utf-8')
# f_mse = open("/public/home/renkan/hwc/exp/chronos-forecasting/forecast_log/Solar_mid_5.jsonl", 'r', encoding='utf-8')
f_pred = open("/public/home/renkan/cty/chronos-forecasting/scripts/test/forecast_log/ETTh1_original_5.jsonl", 'r', encoding='utf-8')
f_gt = open("/public/home/renkan/cty/chronos-forecasting/scripts/test/forecast_log/ETTh1_gt_5.jsonl", 'r', encoding='utf-8')
f_mean = open("/public/home/renkan/cty/chronos-forecasting/scripts/test/forecast_log/ETTh1_mean_5.jsonl", 'r', encoding='utf-8')
f_mse = open("/public/home/renkan/cty/chronos-forecasting/scripts/test/forecast_log/ETTh1_mid_5.jsonl", 'r', encoding='utf-8')

b_pred = open("/public/home/renkan/hwc/exp/chronos-forecasting/forecast_log/custom_patchTST_pred.jsonl", 'r', encoding='utf-8')
b_gt = open("/public/home/renkan/hwc/exp/chronos-forecasting/forecast_log/custom_patchTST_gt.jsonl", 'r', encoding='utf-8')
count = 1
max_num = 0
# for line_mse in f_mse:
# 	if count == 192:
# 		data_mse = json.loads(line_mse)
# 		plt.plot(data_mse)
# 		plt.show()
# 	count += 1
# count = 0
plt.rcParams['savefig.facecolor'] = 'white'
x = np.linspace(0,24,24)
count = 1


for line_pred, line_gt, line_mean, base_line_pred,base_line_gt in zip(f_pred, f_gt, f_mean,b_pred,b_gt):
	data_pred = json.loads(line_pred)
	data_gt = json.loads(line_gt)
	data_mean = json.loads(line_mean)
	data_b   = json.loads(base_line_pred)
	data_b  = np.array(data_b)
	data_bg   = json.loads(base_line_gt)
	data_bg  = np.array(data_bg)
	for i in range(len(data_pred)):
		pred = np.array(data_pred[i])
		data_gt = np.array(data_gt)
		data_b_univr = data_b[:,0]
		data_bg_univr = data_bg[:,0]
        #pred_mean = np.mean(np.abs(pred))
		mae, mse, _, _, _ = metric(pred, data_gt)
		mae_b,mse_b,_,_,_ = metric(data_b_univr,data_gt)
		plt.plot(x,pred,'b',label='prediction')
		plt.plot(x,data_gt,'r',label='gt')
		plt.plot(x,data_b_univr,'g',label='PatchTST')
		# plt.plot(x,data_bg_univr,'y',label='PatchTST_baseline')
		plt.legend()
		plt.title(f"mse_chronos:{mse},mse_PatchTST:{mse_b}")
		plt.savefig(f'./pictures/solar_{count}', dpi=300)
		plt.close()
		count += 1


# for line_pred, line_gt, line_mean in zip(f_pred, f_gt, f_mean):
# 	data_pred = json.loads(line_pred)
# 	data_gt = json.loads(line_gt)
# 	data_mean = json.loads(line_mean)
# 	for i in range(len(data_pred)):
# 		pred = np.array(data_pred[i])
# 		data_gt = np.array(data_gt)
#         #pred_mean = np.mean(np.abs(pred))
# 		mae, mse, _, _, _ = metric(pred, data_gt)
# 		plt.plot(x,pred,'b',label='prediction')
# 		plt.plot(x,data_gt,'r',label='gt')
# 		plt.legend()
# 		plt.savefig(f'./pictures/solar_{count}', dpi=300)
# 		plt.close()
# 		count += 1




# for line_pred, line_gt in zip(b_pred, b_gt):
# 	data_pred = json.loads(line_pred)
# 	data_gt = json.loads(line_gt)
# 	# print("data_pred:",pred.shape)
# 	#print("gt:",data_gt.shape)
# 	pred = np.array(data_pred)
# 	data_gt = np.array(data_gt)

# 	# pred = np.transpose(pred)
# 	# data_gt =np.transpose(data_gt)
# 	#print("pred:",pred.shape)
# 	#print("gt:",data_gt.shape)
# 	#pred_mean = np.mean(np.abs(pred))
# 	mae, mse, _, _, _ = metric(pred, data_gt)
# 	plt.plot(x,pred[:,0],'b',label='prediction')
# 	plt.plot(x,data_gt[:,0],'r',label='gt')
# 	plt.legend()
# 	plt.savefig(f'./pictures/solar_{count}', dpi=300)
# 	plt.close()
# 	count += 1 
		
#     break